## Color Filtering

Check that you have the necessary packages/requirements in order to run this code. Make sure you run the cells in order. 

In [ ]:
# IMPORTS
import cv2
import numpy as np
from sklearn.cluster import DBSCAN

In [ ]:
# Function to filter out everything except the specified color and return the coordinates
# makes everything black that isnt our specified color (zeroes it out)
def filter_and_obtain_coordinates(image, lower_color, upper_color):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_image, lower_color, upper_color)
    result = cv2.bitwise_and(image, image, mask=mask)

    coordinates = np.argwhere(mask > 0)                                     # Finds the colored coordinates (not blacked out)

    return result, coordinates

In [ ]:
# Function to find and return the center coordinates of clusters
# DBSCAN (Density-Based Spatial Clustering of Applications with Noise) algorithm
# identifies clusters and calculates their centers
def get_cluster_centers(coordinates, min_samples=10, eps=10):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coordinates)
    labels = db.labels_

    unique_labels = set(labels) - {-1}                                      # Find the unique labels (besides the noise points (-1))

    cluster_centers = []

    for label in unique_labels:
        cluster_indices = np.where(labels == label)[0]
        cluster_points = coordinates[cluster_indices]

        center = np.mean(cluster_points, axis=0)
        cluster_centers.append(center)

    return cluster_centers

In [ ]:
# START OF CODE

# Choose which color you want to detect
# HSV -> Hue, Saturation, value

# BLUE
# lower_blue = np.array([90, 50, 50])                               # Lower bound for blue in HSV
# upper_blue = np.array([130, 255, 255])                            # Upper bound for blue in HSV

# GREEN
# lower_blue = np.array([35, 50, 50])                               # Lower bound for green in HSV
# upper_blue = np.array([85, 255, 255])                             # Upper bound for green in HSV

# RED
lower_blue = np.array([0, 50, 50])                                  # Lower bound for red in HSV
upper_blue = np.array([10, 255, 255])                               # Upper bound for red in HSV


cap = cv2.VideoCapture(0)                                           # Opens the camera

if not cap.isOpened():                                              # exits if the camera can't be or isn't opened
    print("Error: Could not open the camera.")
    exit()

screenshot = None
screenshot_coordinates = None

                                                                    # Get the frame dimensions (width and height)
frame_width = int(cap.get(3))                                       # 3 corresponds to CV_CAP_PROP_FRAME_WIDTH
frame_height = int(cap.get(4))                                      # 4 corresponds to CV_CAP_PROP_FRAME_HEIGHT


with open("screen_dimensions.txt", "w") as dimensions_file:         # Saves the dimensions of the camera screen to screen_dimensions.txt
    dimensions_file.write(f"Width: {frame_width} pixels\n")         
    dimensions_file.write(f"Height: {frame_height} pixels\n")

while True:                                                         
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read a frame.")
        break


    filtered_frame, color_coordinates = filter_and_obtain_coordinates(frame, lower_blue, upper_blue)        # calls our filter function

    cv2.imshow("Filtered Image", filtered_frame)                    # shows the filtered frame on the screen

    key = cv2.waitKey(1)                                            # waits for a key to be pressed


    if key == ord('q'):                                             # if 'q' is pressed, takes a screenshot 
        screenshot = frame.copy()
        screenshot_coordinates = color_coordinates


        cluster_centers = get_cluster_centers(screenshot_coordinates)   # Get the center coordinates of the clusters


        for center in cluster_centers:                              # loop through a draw green circles at the cluster centers
            center = (int(center[1]), int(center[0]))               # Swap X and Y values (for OpenCV)
            cv2.circle(screenshot, center, 5, (0, 255, 0), -1)



        with open("coordinates.txt", "w") as coord_file:            # Saves all the coordinates to coordinates.txt
            for center in cluster_centers:
                coord_file.write(f"({int(center[1])}, {int(center[0])})\n")

    
        cv2.imwrite("screenshot.png", screenshot)                   # Save the screenshot


        cv2.imwrite("blacked_out.png", filtered_frame)              # Save a screenshot of the filtered image


        print("Screenshot taken and saved as 'screenshot.png'.")    # print confirmation
        print("Coordinates saved to 'coordinates.txt'.")
        print("Blacked-out image saved as 'blacked_out.png'.")
        break


cap.release()                                                       # release the camera 
cv2.destroyAllWindows()                                             # close all OpenCV windows